In [10]:
from DB import DataFrameHandler
from DB import AIModelHandler
import pandas as pd
pd.options.mode.chained_assignment = None

def calcStochasticK(df, n=14):
    Numerator = df - df.rolling(window=n).min()
    Denominator = df.rolling(window=n).max() - df.rolling(window=n).min()
    result = (Numerator / Denominator * 100).round()
    return result

def calcNewsScore(isin, start = '2023/09/01', end = None):
    news = pd.DataFrame([])
    work_dts = AIModelHandler.selectWorkDates(start = start)
    for work_dt in work_dts:
        work_dt = work_dt.date()
        news = pd.concat([news, AIModelHandler.getNewsScore(isin, work_dt)])
    news = news[['ISIN', 'WORK_DT', 'MENTION_SCORE', 'POSITIVE_SCORE']]
    news.fillna(0, inplace=True)
    news['WORK_DT'] = news['WORK_DT'].astype('datetime64[ns]')
    news['POSITIVE_SCORE'] = ((news['POSITIVE_SCORE'] + 100) / 2).round().astype('int')
    return news

def calcPredictScore(isin, start = '2023/09/01', end = None):
    return DataFrameHandler.findItems('predict', columns=["isin, work_dt, PREDICT_PRICE * 10 as price_score, PREDICT_UPDOWN * 10  as updown_score"], condition=f"isin = '{isin}' and work_dt >= TO_DATE('{start}', 'YYYY/MM/DD')")

def calcStatisticsScore(isin, start = '2023/09/01', end = None):
    statistics = DataFrameHandler.findItems('statistics', condition=f"isin = '{isin}' and work_dt >= (SELECT TO_DATE('{start}', 'YYYY/MM/DD') - 20 AS WORK_DT FROM DUAL)")
    statistics['RSI_SCORE'] = calcStochasticK(statistics['RSI'])
    statistics['OBV_SCORE'] = calcStochasticK(statistics['OBV'])
    statistics['STOCHASTIC_SCORE'] = calcStochasticK(statistics['STOCHASTIC_D'])
    statistics['MOMENTUM_SCORE'] = calcStochasticK(statistics['MOMENTUM'])
    statistics = statistics[statistics['WORK_DT']>='2023/09/01'][['ISIN', 'WORK_DT', 'RSI_SCORE', 'OBV_SCORE', 'STOCHASTIC_SCORE', 'MOMENTUM_SCORE']]
    statistics.fillna(0, inplace=True)
    statistics['RSI_SCORE'] = statistics['RSI_SCORE'].astype(int)
    statistics['OBV_SCORE'] = statistics['OBV_SCORE'].astype(int)
    statistics['STOCHASTIC_SCORE'] = statistics['STOCHASTIC_SCORE'].astype(int)
    statistics['MOMENTUM_SCORE'] = statistics['MOMENTUM_SCORE'].astype(int)
    return statistics

def calcScore(isin, start = '2023/09/01', end = None):
    predict = calcPredictScore(isin, start = start, end = end)
    statistics = calcStatisticsScore(isin, start = start, end = end)
    news = calcNewsScore(isin, start = start, end = end)
    result = pd.merge(pd.merge(news, predict, how='outer'), statistics, how='outer') 
    result['AI_SOCRE'] = (result[['PRICE_SCORE', 'UPDOWN_SCORE', 'MENTION_SCORE', 'POSITIVE_SCORE']].sum(axis=1) / 220 * 100)
    result['STATISTICS_SCORE'] = (result[['RSI_SCORE', 'OBV_SCORE', 'STOCHASTIC_SCORE', 'MOMENTUM_SCORE']].sum(axis=1) / 400 * 100)
    result['TOTAL_SCORE'] = (result[['PRICE_SCORE', 'UPDOWN_SCORE', 'MENTION_SCORE', 'POSITIVE_SCORE', 'RSI_SCORE', 'OBV_SCORE', 'STOCHASTIC_SCORE', 'MOMENTUM_SCORE']].sum(axis=1) / 620 * 100)
    result = result.round().astype({'AI_SOCRE': 'int', 'STATISTICS_SCORE': 'int', 'TOTAL_SCORE': 'int'})
    result['WORK_DT'] = result['WORK_DT'].dt.strftime('%Y-%m-%d')
    return result

In [7]:
DataFrameHandler.findItems('stock', columns=['isin'], condition="isin not in (select isin from ONECLE_SCORE where work_dt = (SELECT TO_DATE('2023/09/25', 'YYYY/MM/DD') AS work_dt FROM DUAL))")['ISIN'].to_list()

['014200',
 '015540',
 '019490',
 '024810',
 '030790',
 '032860',
 '033180',
 '033340',
 '038340',
 '001340',
 '003560',
 '089530',
 '091090',
 '093230',
 '096040',
 '099520',
 '101140',
 '102280',
 '106520',
 '109070',
 '111870',
 '114120',
 '117670',
 '121800',
 '121890',
 '044060',
 '047820',
 '052670',
 '053590',
 '056090',
 '057880',
 '058220',
 '058530',
 '062860',
 '005030',
 '006580',
 '007610',
 '008500',
 '012600',
 '257370',
 '263540',
 '268600',
 '269620',
 '290380',
 '299910',
 '318020',
 '065560',
 '066410',
 '068940',
 '069110',
 '078130',
 '136510',
 '138690',
 '141020',
 '148140',
 '150840',
 '160600',
 '174880',
 '178780',
 '181340',
 '208860',
 '214310',
 '214870',
 '215090',
 '217480',
 '219750',
 '226360',
 '226440',
 '372290',
 '373340',
 '377400',
 '388800',
 '391060',
 '393360',
 '404950',
 '426550',
 '427950',
 '435620',
 '435870',
 '436530',
 '438580',
 '448370']

In [21]:
DataFrameHandler.findItems('STATISTICS', condition="isin = '448370'")


,ISIN,WORK_DT,RSI,OBV,STOCHASTIC_K,STOCHASTIC_D,MOMENTUM
0,448370,2023-04-27,83,458087,100,90,45.0
1,448370,2023-04-28,86,479001,100,100,55.0
2,448370,2023-05-02,86,479001,100,100,55.0
3,448370,2023-05-03,88,498552,100,100,65.0
4,448370,2023-05-04,91,523047,100,100,75.0
5,448370,2023-05-08,91,523047,100,100,60.0
6,448370,2023-05-09,95,532850,100,100,85.0
7,448370,2023-05-10,78,517410,90,97,60.0
8,448370,2023-05-11,80,522239,94,95,75.0
9,448370,2023-05-12,71,486690,88,91,60.0


In [18]:
DataFrameHandler.findItems('STATISTICS', columns=['isin, count(*)'], condition="isin != '0' group by isin")

,ISIN,COUNT(*)
0,084180,2284
1,084440,2342
2,086900,2629
3,086670,2629
4,090470,2629
...,...,...
2611,273640,489
2612,274400,42
2613,265560,1445
2614,008490,2629


In [14]:
DataFrameHandler.findItems('STATISTICS', columns='*', condition="work_dt = (SELECT TO_DATE('2023/09/25', 'YYYY/MM/DD') AS work_dt FROM DUAL)")

,ISIN,WORK_DT,RSI,OBV,STOCHASTIC_K,STOCHASTIC_D,MOMENTUM
0,000020,2023-09-25,19,236293824,0,0,-720.0
1,000040,2023-09-25,56,847808942,70,81,73.0
2,000050,2023-09-25,41,7304961,13,5,-40.0
3,000070,2023-09-25,41,1664559,19,17,-200.0
4,000075,2023-09-25,38,460608,38,63,300.0
...,...,...,...,...,...,...,...
2529,950170,2023-09-25,22,16266355,0,0,-400.0
2530,950190,2023-09-25,23,5505679,0,9,-380.0
2531,950200,2023-09-25,25,185046760,0,2,-370.0
2532,950210,2023-09-25,28,100838933,3,14,-680.0


In [11]:
result = calcScore('448370', start = '2023/09/01')
result

,ISIN,WORK_DT,MENTION_SCORE,POSITIVE_SCORE,PRICE_SCORE,UPDOWN_SCORE,RSI_SCORE,OBV_SCORE,STOCHASTIC_SCORE,MOMENTUM_SCORE,AI_SOCRE,STATISTICS_SCORE,TOTAL_SCORE
0,448370,2023-09-01,0,50,0,0,NaN,NaN,NaN,NaN,23,0,8
1,448370,2023-09-04,0,50,0,0,NaN,NaN,NaN,NaN,23,0,8
2,448370,2023-09-05,0,50,0,0,NaN,NaN,NaN,NaN,23,0,8
3,448370,2023-09-06,0,50,0,0,NaN,NaN,NaN,NaN,23,0,8
4,448370,2023-09-07,0,50,0,0,NaN,NaN,NaN,NaN,23,0,8
5,448370,2023-09-08,0,50,0,0,NaN,NaN,NaN,NaN,23,0,8
6,448370,2023-09-11,0,50,0,0,NaN,NaN,NaN,NaN,23,0,8
7,448370,2023-09-12,0,50,0,0,NaN,NaN,NaN,NaN,23,0,8
8,448370,2023-09-13,0,50,0,0,NaN,NaN,NaN,NaN,23,0,8
9,448370,2023-09-14,0,50,0,0,NaN,NaN,NaN,NaN,23,0,8


In [2]:
for stock in AIModelHandler.selectAllStock():
    result = calcScore(stock, start = '2023/09/01')
    DataFrameHandler.insertItems("TMP_ONECLE_SCORE", result)
DataFrameHandler.mergeTable("ONECLE_SCORE", "TMP_ONECLE_SCORE", ['ISIN', 'WORK_DT'])